<a href="https://colab.research.google.com/github/Vinisenso/Multcriteria_Project/blob/main/LE610_Quem_deve_substituir_o_Tite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LE610 - Projeto 2 (Métodos de Apoio à Decisão Multicritério)

Quem deve ser o sucessor do Tite?

# Importanto as bibliotecas e acessando o Drive

In [ ]:
#Importando bibliotecas e acessando o Drive
import pandas as pd
import numpy as np

In [ ]:
!pip install gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Código que importa dados do Google Sheets

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import files

# Criando as funções para o PROMETHEE-II

In [ ]:
# funções de preferência
def f_pref(df, criterio, tipo, func='usual', q=0, p=0):
  n_alt = len(df)
  m = np.zeros([n_alt, n_alt])
  for i in range(0, n_alt):
    for j in range(0, n_alt):
      if tipo == 'max':
        d = df[criterio][i] - df[criterio][j]
      elif tipo == 'min':
        d = df[criterio][j] - df[criterio][i]

      if func == 'usual':
        if d <= 0:
          m[i][j] = 0
        else:
          m[i][j] = 1

      if func == 'U':
        if d <= q:
          m[i][j] = 0
        else:
          m[i][j] = 1

      if func == 'níveis':
        if d <= 0:
          m[i][j] = 0
        elif q < d <= p:
          m[i][j] = 1/2
        else:
          m[i][j] = 1    

      if func == 'linear':
        if d <= q:
          m[i][j] = 0
        elif q < d <= p:
          m[i][j] = (d-q)/(p-q)
        else: 
          m[i][j] = 1

  return m   

# cálculo dos fluxos
def fluxo_pos(M):
  fluxo_pos = []
  for i in range(0, len(M)):
    phi = sum(M[i])/(len(M)-1)
    fluxo_pos.append(phi)
  return np.array(fluxo_pos)

def fluxo_neg(M):
  fluxo_neg = []
  for j in range(0, len(M)):
    phi = sum(M[...,j])/(len(M)-1)
    fluxo_neg.append(phi)
  return np.array(fluxo_neg)

def fluxo_liq(fluxo_pos, fluxo_neg):
  fluxo_liq = fluxo_pos - fluxo_neg
  return fluxo_liq

# Tratamento dos Dados (Matrizes de Decisão D e vetor de pesos)

In [ ]:
# Importando os dados do problema 

d1 = pd.DataFrame(gc.open('LE610 - Projeto 2 (S.A.D.A.M)').get_worksheet(0).get_all_records())
d2 = pd.DataFrame(gc.open('LE610 - Projeto 2 (S.A.D.A.M)').get_worksheet(1).get_all_records())
d3 = pd.DataFrame(gc.open('LE610 - Projeto 2 (S.A.D.A.M)').get_worksheet(2).get_all_records())
d4 = pd.DataFrame(gc.open('LE610 - Projeto 2 (S.A.D.A.M)').get_worksheet(3).get_all_records())

In [ ]:
# extraindo os vetores de pesos:
def pesos(df):
  df = df.set_index('Alternativas')
  df.index.names = [None]

  peso = df.loc['Pesos']
  peso = list(np.array(peso))

  w = []
  for i in range(len(peso)):
    peso[i] = peso[i].replace(',', '.')
    w.append(float(peso[i]))

  return w

w_d1 = pesos(d1)
w_d2 = pesos(d2)
w_d3 = pesos(d3)
w_d4 = pesos(d4)

In [ ]:
# tornando as alternativas como índices
# mudando os nomes das colunas
# transformando as avaliações qualitativas em números
def manipula(df):
  df = df.set_index('Alternativas')
  df.index.names = [None]

  colunas = ['exp', 'tit', 'dur', 'GC', 'GP', 'aprov', 'aceit', 'est']
  df.columns = colunas
  
  df = df.drop(df.index[len(df)-1])

  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['aceit'] = df['aceit'].map(dic)
  df['est'] = df['est'].map(dic)

  # tratando os dados numéricos (transformando , em .)
  col = ['dur', 'GC', 'GP', 'aprov']
  for i in colunas:
    df[i] = df[i].astype(str)
    df[i] = df[i].str.replace(',', '.')
    df[i] = df[i].astype(float)

  return df

d1 = manipula(d1)
d2 = manipula(d2)
d3 = manipula(d3)
d4 = manipula(d4)

In [ ]:
# printando para ver o resultado final do tratamento
d1

,exp,tit,dur,GC,GP,aprov,aceit,est
Dorival Júnior,20.0,12.0,5.7,1.5,1.0,55.89,4.0,3.0
Cuca,24.0,14.0,7.0,1.5,1.0,56.22,4.0,3.0
Fernando Diniz,13.0,0.0,6.0,1.3,1.2,50.56,3.0,5.0
Mano Menezes,25.0,11.0,8.0,1.6,1.2,55.33,3.0,2.0
Rogério Ceni,6.0,7.0,8.7,1.8,1.2,58.00,3.0,3.0
Maurício Barbieri,12.0,0.0,10.3,1.0,1.5,28.11,2.0,2.0
Jorginho,17.0,4.0,6.3,1.4,1.2,49.00,1.0,1.0
Dunga,16.0,3.0,11.3,1.9,1.0,66.89,1.0,1.0


# Resolvendo com PROMETHEE-II

In [ ]:
# criando função para calcular a matriz de preferência M
def M(df, w):
  m_exp = f_pref(df, 'exp', 'max')
  m_tit = f_pref(df, 'tit', 'max')
  m_dur = f_pref(df, 'dur', 'max')
  m_GC = f_pref(df, 'GC', 'min')
  m_GP = f_pref(df, 'GP', 'max')
  m_aprov = f_pref(df, 'aprov', 'max')
  m_aceit = f_pref(df, 'aceit', 'max')
  m_est = f_pref(df, 'est', 'max')

  matrizes = [m_exp, m_tit, m_dur, m_GC, m_GP, m_aprov, m_aceit, m_est]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# cálculo das matrizes de preferência para cada decisor:
M_d1 = M(d1, w_d1)
M_d2 = M(d2, w_d2)
M_d3 = M(d3, w_d3)
M_d4 = M(d4, w_d4)

In [ ]:
M_d1

array([[0.  , 0.  , 0.63, 0.74, 0.53, 0.82, 0.82, 0.72],
       [0.67, 0.  , 0.69, 0.74, 0.53, 0.82, 0.88, 0.72],
       [0.36, 0.3 , 0.  , 0.22, 0.33, 0.45, 0.37, 0.32],
       [0.25, 0.25, 0.67, 0.  , 0.51, 0.63, 0.88, 0.8 ],
       [0.27, 0.27, 0.56, 0.38, 0.  , 0.71, 0.77, 0.69],
       [0.17, 0.17, 0.17, 0.17, 0.28, 0.  , 0.38, 0.32],
       [0.17, 0.11, 0.54, 0.03, 0.14, 0.61, 0.  , 0.59],
       [0.19, 0.19, 0.67, 0.19, 0.3 , 0.67, 0.19, 0.  ]])

In [ ]:
# obtenção dos fluxos líquidos
fl_d1 = fluxo_liq(fluxo_pos(M_d1), fluxo_neg(M_d1))
fl_d2 = fluxo_liq(fluxo_pos(M_d2), fluxo_neg(M_d2))
fl_d3 = fluxo_liq(fluxo_pos(M_d3), fluxo_neg(M_d3))
fl_d4 = fluxo_liq(fluxo_pos(M_d4), fluxo_neg(M_d4))

In [ ]:
# função para ordenamento
def ordenamento(decisor, df, fl):
  # gerando um dicionário para ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

print('Augusto:')
print(ordenamento(1, d1, fl_d1))
print(80*'-')
print('João:')
print(ordenamento(2, d2, fl_d2))
print(80*'-')
print('Nathan:')
print(ordenamento(3, d3, fl_d3))
print(80*'-')
print('Vinícius:')
print(ordenamento(4, d4, fl_d4))
print(80*'-')

Augusto:
                   Fluxo Líquido  Rank D1
Cuca                    0.537143        1
Dorival Júnior          0.311429        2
Mano Menezes            0.217143        3
Rogério Ceni            0.147143        4
Fernando Diniz         -0.225714        5
Dunga                  -0.251429        6
Jorginho               -0.300000        7
Maurício Barbieri      -0.435714        8
--------------------------------------------------------------------------------
João:
                   Fluxo Líquido  Rank D2
Cuca                    0.462857        1
Mano Menezes            0.291429        2
Dunga                   0.238571        3
Dorival Júnior          0.214286        4
Jorginho               -0.162857        5
Maurício Barbieri      -0.232857        6
Rogério Ceni           -0.360000        7
Fernando Diniz         -0.451429        8
--------------------------------------------------------------------------------
Nathan:
                   Fluxo Líquido  Rank D3
Dorival Júnior   

In [ ]:
# gerando as tabelas para cada decisor
ordem_d1 = ordenamento(1, d1, fl_d1)
ordem_d2 = ordenamento(2, d2, fl_d2)
ordem_d3 = ordenamento(3, d3, fl_d3)
ordem_d4 = ordenamento(4, d4, fl_d4)


# criando um dataframe com os ordenamento de todos os decisores:
ordem = pd.concat([ordem_d1, ordem_d2, ordem_d3, ordem_d4], axis=1)
ordem
ordem = ordem.drop(['Fluxo Líquido'], axis=1)
ordem

,Rank D1,Rank D2,Rank D3,Rank D4
Cuca,1,1,2,2
Dorival Júnior,2,4,1,3
Mano Menezes,3,2,7,1
Rogério Ceni,4,7,4,8
Fernando Diniz,5,8,3,7
Dunga,6,3,6,5
Jorginho,7,5,8,4
Maurício Barbieri,8,6,5,6


In [ ]:
# criando uma escala para borda
borda = {}
for i in range(0, len(ordem)):
  borda.update({i+1: (len(ordem)-i)})

# substituindo os valores de ordenamento pelo valor da escala de Borda
# Ex: (1,2,3) vai virar (3,2,1)
for i in ordem.columns:
  ordem[i] = ordem[i].map(borda)

# somando os valores de cada alternativa
ordem['Valor'] = ordem.sum(axis=1)

ordem_final = pd.DataFrame(ordem['Valor'])
ordem_final = ordem_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(ordem_final)):
  ord.append(f'{i+1}°')

ordem_final['Ordem Final (Borda)'] = ord
ordem_final

,Valor,Ordem Final (Borda)
Cuca,30,1°
Dorival Júnior,26,2°
Mano Menezes,23,3°
Dunga,16,4°
Rogério Ceni,13,5°
Fernando Diniz,13,6°
Jorginho,12,7°
Maurício Barbieri,11,8°


###Resolvendo com Topsis

In [ ]:
def topsis(decisor,df,w):
  #Normalização Euclidiana
  
  dfn=df.copy()
  v= df.to_numpy() # Transformando um Data Frame em array visto que é um formato melhor para se trabalhar com dados numéricos e se fazer contas.
  v=v.transpose()  # Transpondo o vetor visto que os dados interessantes que se precisa somar e integralizar estão em colunas, logo transformo as colunas em linhas e, assim se consegue fazer alguns tratamentos, como for exemplo cálcular a norma dos dados da antiga coluna.

  c=0 #adiciono um contador para controlar o índice numérico de qual coluna se está analisando no laço for a seguir.
  
  for coluna in d1.columns:  
    for linha in d1.index:
      if coluna== "GC":
        dfn[coluna][linha]= 1-d1[coluna][linha]/np.linalg.norm(v[c]) # para cada dado da coluna, se realiza a normalização Euclidiana
      else:
         dfn[coluna][linha]= d1[coluna][linha]/np.linalg.norm(v[c])
    c+=1
  
  #aplicando os pesos
  
  vn=dfn.to_numpy() # transforma-se os dados normalizados de Data Frame para vetor para se fazer uma multiplicação vetorial
  w_1=np.array(w)  # transforma-se a lista de pesos em vetor para que se possa fazer conta com ele. 
  w_1=w_1.reshape(len(w_1),1) # faz-se um "reshape" a fim de permitir que o vetor tenha o mesmo número de linhas que o número de colunas da matriz de valores normalizados.

  for i in range(0, len(vn)):
    for j in range(0,len(w_1)):
      vn[i][j]=vn[i][j]*w_1[j][0] #faço a multiplicação vetorial (pode-se tentar usar a função np.matmul mas não deu muito certo a multiplicação, então fiz no modo viking)
  
  #Fazendo a lista com os valores mínimos e máximos de cada critério
  lmax=[]
  lmin=[] # a partir destas duas listas, armazenará-se os menores e maioires valores que cada critério possui, t- e t+, respectivamente. 
  for j in range(len(vn.transpose())):
    lmax.append(max(vn.transpose()[j])) #peço os menores e maiores valores da matriz vn já com os pesos aplicados e um detalhe importante é que novamente se transpõe a matriz para que se possa analisar as colunas por meio das funções do python.
    lmin.append(min(vn.transpose()[j]))

  #fazendo as distâncias
  
  nota=[] #lista que vai armazenar as notas finais atribuidas pelo topsis
  dma=0 # criação das variáveis distância maior (dma) e distância menor (dme) para receberem as somas das notas de cada alternativa em cada critério
  dme=0
  for i in range(0,len(vn)):
    for j in range(0,len(lmax)):
      dma+=(lmax[j]-vn[i][j])**2
      dme+=(lmin[j]-vn[i][j])**2
    nota.append(dme**(1/2)/(dme**(1/2)+dma**(1/2))) #após a noma calcula-se a nota final d-/(d+ + d-)
    dma=0
    dme=0

  dff=pd.DataFrame(nota,index=df.index,columns=["Nota Final"])
  dff=dff.sort_values(by=['Nota Final'],ascending=False)
  
  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  dff[f'Rank D{decisor}'] = ord

  return dff





In [ ]:
print('Augusto:')
print(topsis(1, d1, w_d1))
print(80*'-')
print('João:')
print(topsis(2, d2, w_d2))
print(80*'-')
print('Nathan:')
print(topsis(3, d3, w_d3))
print(80*'-')
print('Vinícius:')
print(topsis(4, d4, w_d4))
print(80*'-')

Augusto:
                   Nota Final  Rank D1
Cuca                 0.818213        1
Dorival Júnior       0.762985        2
Mano Menezes         0.676037        3
Rogério Ceni         0.490932        4
Fernando Diniz       0.306184        5
Jorginho             0.273024        6
Dunga                0.240652        7
Maurício Barbieri    0.121212        8
--------------------------------------------------------------------------------
João:
                   Nota Final  Rank D2
Cuca                 0.730976        1
Mano Menezes         0.655904        2
Dorival Júnior       0.653737        3
Fernando Diniz       0.540290        4
Dunga                0.432213        5
Jorginho             0.412867        6
Rogério Ceni         0.327570        7
Maurício Barbieri    0.275988        8
--------------------------------------------------------------------------------
Nathan:
                   Nota Final  Rank D3
Fernando Diniz       0.658817        1
Cuca                 0.606900      

In [ ]:
# gerando as tabelas para cada decisor
ordemt_d1 = topsis(1, d1, w_d1)
ordemt_d2 = topsis(2, d2, w_d2)
ordemt_d3 = topsis(3, d3, w_d3)
ordemt_d4 = topsis(4, d4, w_d4)


# criando um dataframe com os ordenamento de todos os decisores:
ordemt = pd.concat([ordemt_d1, ordemt_d2, ordemt_d3, ordemt_d4], axis=1)
ordemt = ordemt.drop(['Nota Final'], axis=1)
ordemt

,Rank D1,Rank D2,Rank D3,Rank D4
Cuca,1,1,2,2
Dorival Júnior,2,3,3,3
Mano Menezes,3,2,5,1
Rogério Ceni,4,7,4,8
Fernando Diniz,5,4,1,6
Jorginho,6,6,7,4
Dunga,7,5,6,5
Maurício Barbieri,8,8,8,7


In [ ]:
# criando uma escala para borda
borda = {}
for i in range(0, len(ordemt)):
  borda.update({i+1: (len(ordemt)-i)})

# substituindo os valores de ordenamento pelo valor da escala de Borda
# Ex: (1,2,3) vai virar (3,2,1)
for i in ordemt.columns:
  ordemt[i] = ordemt[i].map(borda)

# somando os valores de cada alternativa
ordemt['Valor'] = ordemt.sum(axis=1)

ordemt_final = pd.DataFrame(ordemt['Valor'])
ordemt_final = ordemt_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(ordemt_final)):
  ord.append(f'{i+1}°')

ordemt_final['Ordem Final (Borda)'] = ord
ordemt_final

,Valor,Ordem Final (Borda)
Cuca,30,1°
Dorival Júnior,25,2°
Mano Menezes,25,3°
Fernando Diniz,20,4°
Rogério Ceni,13,5°
Jorginho,13,6°
Dunga,13,7°
Maurício Barbieri,5,8°
